<a href="https://colab.research.google.com/github/hargagan/EDA-NYC-Taxi-Data-Analysis/blob/main/pyspark/C3_M2_Practice_exercise_assgnmnt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('chipotle').getOrCreate()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_chipotle = spark.read.csv('/content/drive/MyDrive/Assignments/EDA/chipotle (1).tsv', sep='\t', header=True)

In [7]:
df_chipotle.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows


#### **Task 1:** Write a function that returns the total revenue generated from all orders. Hint: Multiply quantity by item_price for each row and then add the results.

In [10]:
def get_total_revenue(df):
    from pyspark.sql.functions import udf, regexp_replace, trim
    from pyspark.sql.types import FloatType
    # Remove '$' and trim spaces before casting to float
    df = df.withColumn('item_price', trim(regexp_replace(df['item_price'], '\\$', '')).cast('float'))
    df = df.withColumn('revenue', df['quantity'] * df['item_price'])
    return df.agg({'revenue': 'sum'}).collect()[0][0]

get_total_revenue(df_chipotle)

39237.01973223686

####**Task 2:** Write a function that returns the most frequently ordered item. Hint: Group by item_name and sum the quantity.

In [13]:
def get_most_ordered_item(df):
    from pyspark.sql.functions import sum
    return df.groupBy('item_name').agg(sum('quantity').alias('total_quantity')).orderBy('total_quantity', ascending=False).first()


get_most_ordered_item(df_chipotle)


Row(item_name='Chicken Bowl', total_quantity=761.0)

####**Task 3:** Write a function that returns the total number of unique items sold. Hint: Count the number of distinct values in the item_name column.

In [14]:
def count_distinct_items(df):
    return df.select('item_name').distinct().count()

count_distinct_items(df_chipotle)

50

####**Task 4:** Write a function that returns the top 5 items by total revenue. Hint: Multiply quantity and item_price, group by item_name, and sort in descending order.

In [16]:
def get_top_5_items_by_revenue(df):
    from pyspark.sql.functions import regexp_replace, trim

    df = df.withColumn('item_price', trim(regexp_replace(df['item_price'], '\\$', '')).cast('float'))
    df = df.withColumn('revenue', df['quantity'] * df['item_price'])
    df.createOrReplaceTempView('chipotle')
    return spark.sql("""
        SELECT item_name, SUM(revenue) AS total_revenue
        FROM chipotle
        GROUP BY item_name
        ORDER BY total_revenue DESC
        LIMIT 5
    """)

get_top_5_items_by_revenue(df_chipotle).show()
#

+-------------------+------------------+
|          item_name|     total_revenue|
+-------------------+------------------+
|       Chicken Bowl|8044.6299295425415|
|    Chicken Burrito| 6387.059944152832|
|      Steak Burrito| 4236.129955291748|
|         Steak Bowl|2479.8099822998047|
|Chips and Guacamole| 2475.619917869568|
+-------------------+------------------+



####**Task 5:** Write a function that returns the average number of items per order. Hint: Group by order_id, sum the quantity per order, and calculate the mean.

In [17]:
def get_average_items_per_order(df):
    from pyspark.sql.functions import avg
    df.createOrReplaceTempView('chipotle')
    return spark.sql("""
        SELECT AVG(total_quantity) AS avg_items_per_order
        FROM (
            SELECT order_id, SUM(quantity) AS total_quantity
            FROM chipotle
            GROUP BY order_id
        ) AS order_quantities
    """).collect()[0][0]

get_average_items_per_order(df_chipotle)
#

2.711014176663032

####**Task 6:** List all beverages sold and their total revenue.

In [25]:
def list_beverages_and_revenue(df):
  from pyspark.sql.functions import regexp_replace, trim, sum, round
  df = df.withColumn('item_price', trim(regexp_replace(df['item_price'], '\\$', '')).cast('float'))
  df = df.withColumn('revenue', df['quantity'] * df['item_price'])
  return df.filter(df['item_name'].isin(['6 Pack Soft Drink', 'Canned Soft Drink', 'Izze'])).groupBy('item_name').agg(round(sum('revenue'), 2).alias('total_revenue')).show()

list_beverages_and_revenue(df_chipotle)


+-----------------+-------------+
|        item_name|total_revenue|
+-----------------+-------------+
|6 Pack Soft Drink|       369.93|
|Canned Soft Drink|       603.75|
|             Izze|         67.8|
+-----------------+-------------+

